# MACs

In [1]:
import torch
from torch import nn

## MACsを計算する

- 畳み込み層のMACsを計算する関数を作成
    - nn.Conv2dのインスタンスと入力shapeを引数に取る
        - .weight.shape()で重みのshapeを取得
- 全結合層のMACsを計算する関数を作成
  - nn.Linearのインスタンスと入力shapeを引数に取る
- 出力のshapeを計算する関数を作成
    - Layerインスタンスと入力tensorを使って順伝播させることを計算する

In [2]:
# LayerとInput　sizeを受け取ったらshapeを返す関数
def calc_output_shape(layer, input_shape):
    input = torch.randn(input_shape)
    output = layer(input)
    return tuple(output.shape)


# 畳み込み層のMACsを計算する関数
def calc_macs_conv2d(layer, input_shape):
    b, in_ch, in_h, in_w= input_shape
    out_ch, _, f_h, f_w = layer.weight.shape

    _, _, out_h, out_w = calc_output_shape(layer, input_shape)

    macs = b * in_ch * out_ch * f_h * f_w * out_h * out_w

    return macs


# 全結合層のMACsを計算する関数
def calc_macs_linear(layer, input_shape):
    b, in_features= input_shape
    out_features, _ = layer.weight.shape

    macs = b * in_features * out_features

    return macs

In [3]:
input_shape = (1, 1, 128, 128)
X = torch.randn(input_shape)
# 畳み込み層
conv_layer = nn.Conv2d(1, 8, kernel_size=3)
calc_macs_conv2d(conv_layer, input_shape)

1143072

In [4]:
# 全結合層
linear_layer = nn.Linear(64, 10)
input_shape = (1, 64)
calc_macs_linear(linear_layer, input_shape)

640

## thopライブラリで計算する

thop.profile()

- thopライブラリを使用することで簡単にMACsを計算できる
- thop.profile():
    - model引数にpytorchのモデルを渡す
    - input引数に入力tensorを(X,y)の形でtupleで渡す
        - 通常のmodelはXのみを入力と取るので、(X, )のようにして渡す
    - MACsおよびパラメータの総数を返す

In [5]:
# !pip install thop
import thop

In [6]:
# conv2d
macs, params = thop.profile(conv_layer, (X,))
# スクラッチで求めたMACsと等しい
macs

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.


1143072.0

In [7]:
# linear
X = torch.randn(input_shape)
macs, params = thop.profile(linear_layer, (X,))
# スクラッチの関数で求めたMACsと等しい
macs

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


640.0